### 1. Load the Fish Data, split into train and test set, and standardize data.

In [ ]:
import pandas as pd
fish_data = pd.read_csv('fish2.csv')
fish_data

In [ ]:
## Create Dummy Variables and convert to torch
import torch
X = ...
X = ...
y = ...

In [ ]:
## Split into train and test set
generator = torch.Generator().manual_seed(42)
idx = torch.randperm(X.shape[0], generator=generator)
X_train, X_test = X[idx[:127],:], X[idx[127:],:]
y_train, y_test = y[idx[:127],:], y[idx[127:],:]

In [ ]:
## Standardize Data
# Lambda Function
standardize = lambda z, z_mean, z_std: (z - z_mean) / (z_std + 1e-5) 

# Standardize X
x_mean, x_std =...
...
# Standardize y
y_mean, y_std = ...
...

### 2 Create k Nearest Neighbor Regressor class

In [ ]:
class kNNRegressor:
    # k: Number of k Nearest Neighbors
    # p: L_p distance function
    # Note: output label is simple average of k Nearest Neighbors
    def __init__(self, k, p=2.0):
        self.k = k
        self.p = p
        self.train_x = None
        self.train_y = None
    
    def fit(self, X, y):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """  
        ...
        return self
        
    def predict(self, X, mean=None, std=None):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        """
        # Calculate distance matrix
        dist = ...
        dist = dist.pow(self.p).mean(2).pow(1./self.p)
        
        # Get k nearest neighbots
        _, idx_arg = ...
        
        # Collect labels
        train_y = ... # First repeat train labels to match idx_arg
        predictions = ... # Next, gather train labels
        predictions = ... # Finally, calculate mean
        
        # Recompute unstandardized data
        if mean is not None and std is not None:
            predictions = ...
        return predictions

In [ ]:
k, p = 5, 2.
knn_model = kNNRegressor(k, p=p).fit(X_train, y_train)
y_pred = knn_model.predict(X_test, y_mean, y_std)
mse = ...
print(f'The MSE of the {k}-NN model with L_{p}-Distance is: {mse:10.4f}')

### 3. Create Linear Regressor class

In [ ]:
class LinearRegression:
    def __init__(self):
        self.weight = None
            
    def fit(self, X, y):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """        
        
        # prepend a column of ones
        ones = ...
        X = ...       

        # compute weights
        XtX_pinv = ...
        Xty = ...
        
        # dot: matrix multiplication
        self.weight = ...
        return self
                
    def predict(self, X, mean=None, std=None):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        """                     
        # prepend a column of ones
        ones = ...
        X = ...

        # compute predictions
        predictions = ...
        
        # Recompute unstandardized data
        if mean is not None and std is not None:
            predictions = ...
        return predictions

In [ ]:
lr_model = LinearRegression().fit(X_train, y_train)
y_pred = lr_model.predict(X_test, y_mean, y_std)
mse = ...
print(f'The MSE of the linear regression model is: {mse:10.4f}')

### 4. Create a Linear Regressor class, but this time use PyTorch autograd function to calculate the gradient.

In [ ]:
import torch.nn as nn

class LinearRegressionGradientDescent:
    def __init__(self, input_features, lr=0.1, epochs=1000):
        self.weight = nn.Parameter(torch.zeros(input_features + 1, 1))
        self.lr, self.epochs = lr, epochs
        
    def fit(self, X, y):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """        
        for _ in range(self.epochs):
            y_pred = ...
            loss = ...
            loss.backward()
            with torch.no_grad():
                self.weight -= ...
            self.weight.grad = None
        return self
    
    def forward(self, X):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        """                     
        # prepend a column of ones
        ones = ...
        X = ...

        # compute predictions
        predictions = ...

        return predictions
    
    def predict(self, X, mean=None, std=None):
        # Execute forward pass
        predictions = ...
        
        # Recompute unstandardized data
        if mean is not None and std is not None:
            predictions = ...
            
        return predictions

In [ ]:
lrd_model = LinearRegressionGradientDescent(12).fit(X_train, y_train)
y_pred = lrd_model.predict(X_test, y_mean, y_std)
mse = ...
print(f'The MSE of the gradient descent linear regression model is: {mse:10.4f}')